In [2]:
import os
import glob
import pandas as pd
from math import pi
import matplotlib.pyplot as plt

In [3]:
path_results = 'results/'
results_files = glob.glob(f'{path_results}/*/*/*.csv')

In [4]:
cenario_files = [f.split('\\')[1] for f in results_files]
repeticao_files = [f.split('\\')[2] for f in results_files]
names_files = [os.path.basename(f) for f in results_files]
todos_files_categorias = list(zip(cenario_files, repeticao_files, names_files, results_files))

In [5]:
df = pd.DataFrame(
    todos_files_categorias,
    columns=["Cenário", "Repetição", "Nome do Arquivo", "Caminho Completo"]
)
df.head()

,Cenário,Repetição,Nome do Arquivo,Caminho Completo
0,CenarioA,rep1,results_exceptions.csv,results\CenarioA\rep1\results_exceptions.csv
1,CenarioA,rep1,results_failures.csv,results\CenarioA\rep1\results_failures.csv
2,CenarioA,rep1,results_stats.csv,results\CenarioA\rep1\results_stats.csv
3,CenarioA,rep1,results_stats_history.csv,results\CenarioA\rep1\results_stats_history.csv
4,CenarioA,rep10,results_exceptions.csv,results\CenarioA\rep10\results_exceptions.csv


In [10]:
path_results = "results"

def safe_read_csv(path):
    try:
        return pd.read_csv(path, encoding='utf-8')
    except:
        try:
            return pd.read_csv(path, encoding='latin-1')
        except:
            return pd.DataFrame()

def filter_valid_timestamps(df):
    if 'Timestamp' in df.columns and not df.empty:
        if pd.api.types.is_numeric_dtype(df['Timestamp']):
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s', errors='coerce')
        else:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
        df = df.dropna(subset=['Timestamp'])
    return df

def remove_warmup(df, warmup_seconds=60):
    if 'Timestamp' in df.columns and not df.empty:
        start_time = df['Timestamp'].min()
        df = df[df['Timestamp'] >= start_time + pd.Timedelta(seconds=warmup_seconds)]
    return df

dados_processados = {}

for cenario in df["Cenário"].unique():
    dados_processados[cenario] = {'repeticoes': {}, 'mediana': {}}
    grupo = df[df["Cenário"] == cenario]
    
    for repeticao, subset in grupo.groupby("Repetição"):
        dados_rep = {'exceptions': None, 'failures': None, 'stats_history': None, 'stats': None}
        
        for _, row in subset.iterrows():
            nome = str(row["Nome do Arquivo"]).lower()
            caminho = row["Caminho Completo"]
            d = safe_read_csv(caminho)
            
            if d.empty:
                continue
            
            if 'exceptions' in nome and 'Message' in d.columns:
                dados_rep['exceptions'] = pd.concat([dados_rep['exceptions'], d], ignore_index=True) if dados_rep['exceptions'] is not None else d
            
            elif 'failures' in nome and 'Error' in d.columns:
                dados_rep['failures'] = pd.concat([dados_rep['failures'], d], ignore_index=True) if dados_rep['failures'] is not None else d
            
            elif 'stats_history' in nome:
                d = filter_valid_timestamps(d)
                d = remove_warmup(d)
                if not d.empty:
                    dados_rep['stats_history'] = pd.concat([dados_rep['stats_history'], d], ignore_index=True) if dados_rep['stats_history'] is not None else d
            
            elif 'stats' in nome and 'Name' in d.columns and 'Average Response Time' in d.columns:
                dados_rep['stats'] = pd.concat([dados_rep['stats'], d], ignore_index=True) if dados_rep['stats'] is not None else d
        
        dados_processados[cenario]['repeticoes'][repeticao] = dados_rep
    
    files_history = grupo[grupo["Nome do Arquivo"].str.contains('stats_history', case=False)]["Caminho Completo"].tolist()
    
    if files_history:
        dfs = []
        for f in files_history:
            d = safe_read_csv(f)
            d = filter_valid_timestamps(d)
            d = remove_warmup(d)
            if not d.empty:
                d['_repeticao'] = f
                dfs.append(d)
        
        if dfs:
            df_hist_all = pd.concat(dfs, ignore_index=True)
            cols = [c for c in ['Requests/s', 'Failures/s'] if c in df_hist_all.columns]
            
            if cols:
                df_hist_all = df_hist_all.sort_values(['_repeticao', 'Timestamp'])
                df_hist_all['_tempo_relativo'] = df_hist_all.groupby('_repeticao')['Timestamp'].transform(
                    lambda x: (x - x.min()).dt.total_seconds()
                )
                df_hist_all['_tempo_bin'] = (df_hist_all['_tempo_relativo'] / 1).astype(int)
                df_median = df_hist_all.groupby('_tempo_bin')[cols].median().reset_index().sort_values('_tempo_bin')
                
                dados_processados[cenario]['mediana']['df_hist_all'] = df_hist_all
                dados_processados[cenario]['mediana']['df_median'] = df_median
    
    files_stats = grupo[(grupo["Nome do Arquivo"].str.contains('stats', case=False)) & (~grupo["Nome do Arquivo"].str.contains('history', case=False))]["Caminho Completo"].tolist()
    if files_stats:
        dfs_stats = [safe_read_csv(f) for f in files_stats]
        dfs_stats = [d for d in dfs_stats if not d.empty and 'Name' in d.columns and 'Average Response Time' in d.columns]
        if dfs_stats:
            dados_processados[cenario]['mediana']['stats'] = pd.concat(dfs_stats, ignore_index=True)
            

In [7]:
for cenario, dados in dados_processados.items():
    for repeticao, dados_rep in dados['repeticoes'].items():
        plots_dir = os.path.join(path_results, 'plots', cenario, str(repeticao))
        os.makedirs(plots_dir, exist_ok=True)
        
        if dados_rep['exceptions'] is not None:
            fig, ax = plt.subplots(figsize=(12, 6))
            dados_rep['exceptions']['Message'].value_counts().nlargest(20).plot(kind='barh', ax=ax, color='salmon')
            ax.set_title(f'Exceções - {cenario} Rep{repeticao}')
            ax.grid(True, alpha=0.3)
            fig.tight_layout()
            fig.savefig(os.path.join(plots_dir, 'excecoes.png'), dpi=150)
            plt.close(fig)
        
        if dados_rep['failures'] is not None:
            fig, ax = plt.subplots(figsize=(12, 6))
            dados_rep['failures']['Error'].value_counts().nlargest(20).plot(kind='barh', ax=ax, color='coral')
            ax.set_title(f'Falhas - {cenario} Rep{repeticao}')
            ax.grid(True, alpha=0.3)
            fig.tight_layout()
            fig.savefig(os.path.join(plots_dir, 'falhas.png'), dpi=150)
            plt.close(fig)
        
        if dados_rep['stats_history'] is not None:
            df_hist = dados_rep['stats_history'].sort_values('Timestamp')
            cols = [c for c in ['Requests/s', 'Failures/s'] if c in df_hist.columns]
            
            if cols:
                fig, ax = plt.subplots(figsize=(12, 6))
                for col in cols:
                    ax.plot(range(len(df_hist)), df_hist[col], label=col, alpha=0.7)
                ax.set_title(f'Histórico - {cenario} Rep{repeticao}')
                ax.set_xlabel('Amostras')
                ax.legend()
                ax.grid(True, alpha=0.3)
                fig.tight_layout()
                fig.savefig(os.path.join(plots_dir, 'historico.png'), dpi=150)
                plt.close(fig)
                
                # BOXPLOT SEPARADO EM DOIS GRÁFICOS
                fig, axes = plt.subplots(1, 2, figsize=(12, 6))
                
                for i, col in enumerate(cols):
                    bp = axes[i].boxplot([df_hist[col].dropna().values], 
                                        labels=[col], 
                                        patch_artist=True)
                    color = 'lightblue' if 'Requests' in col else 'lightcoral'
                    bp['boxes'][0].set_facecolor(color)
                    axes[i].set_title(f'{col}')
                    axes[i].grid(True, alpha=0.3)
                
                fig.suptitle(f'Distribuição - {cenario} Rep{repeticao}', fontsize=14, y=1.02)
                fig.tight_layout()
                fig.savefig(os.path.join(plots_dir, 'boxplot.png'), dpi=150)
                plt.close(fig)
        
        if dados_rep['stats'] is not None:
            fig, ax = plt.subplots(figsize=(12, 6))
            dados_rep['stats'].groupby('Name')['Average Response Time'].mean().nlargest(30).plot(kind='barh', ax=ax, color='steelblue')
            ax.set_title(f'Tempo Médio por Endpoint - {cenario} Rep{repeticao}')
            ax.set_xlabel('ms')
            ax.grid(True, alpha=0.3)
            fig.tight_layout()
            fig.savefig(os.path.join(plots_dir, 'endpoints.png'), dpi=150)
            plt.close(fig)
    
    if dados['mediana']:
        plots_dir = os.path.join(path_results, 'plots', cenario, 'mediana')
        os.makedirs(plots_dir, exist_ok=True)
        
        if 'df_median' in dados['mediana']:
            df_median = dados['mediana']['df_median']
            cols = [c for c in ['Requests/s', 'Failures/s'] if c in df_median.columns]
            
            fig, ax = plt.subplots(figsize=(12, 4))
            for col in cols:
                ax.plot(df_median['_tempo_bin'], df_median[col], label=col)
            ax.set_title(f'Histórico Mediano - {cenario}')
            ax.set_xlabel('Segundos')
            ax.legend()
            ax.grid(True, alpha=0.3)
            fig.tight_layout()
            fig.savefig(os.path.join(plots_dir, 'historico_mediana.png'), dpi=150)
            plt.close(fig)
        
        if 'df_hist_all' in dados['mediana']:
            df_hist_all = dados['mediana']['df_hist_all']
            cols = [c for c in ['Requests/s', 'Failures/s'] if c in df_hist_all.columns]
            
            if cols:
                fig, axes = plt.subplots(1, len(cols), figsize=(6*len(cols), 6))
                if len(cols) == 1:
                    axes = [axes]
                
                reps = sorted(df_hist_all['_repeticao'].unique())
                for ax, col in zip(axes, cols):
                    bp = ax.boxplot([df_hist_all[df_hist_all['_repeticao']==r][col].dropna().values for r in reps],
                                    labels=range(1, len(reps)+1), patch_artist=True)
                    for patch in bp['boxes']:
                        patch.set_facecolor('lightblue')
                    ax.set_title(f'{col} por Repetição')
                    ax.set_xlabel('Repetição')
                    ax.grid(True, alpha=0.3)
                
                fig.tight_layout()
                fig.savefig(os.path.join(plots_dir, 'boxplot_por_repeticao.png'), dpi=150)
                plt.close(fig)
                
                # BOXPLOT GERAL TAMBÉM SEPARADO
                fig, axes = plt.subplots(1, 2, figsize=(12, 6))
                
                for i, col in enumerate(cols):
                    bp = axes[i].boxplot([df_hist_all[col].dropna().values], 
                                        labels=[col], 
                                        patch_artist=True)
                    color = 'lightblue' if 'Requests' in col else 'lightcoral'
                    bp['boxes'][0].set_facecolor(color)
                    axes[i].set_title(f'{col}')
                    axes[i].grid(True, alpha=0.3)
                
                fig.suptitle(f'Distribuição Geral - {cenario}', fontsize=14)
                fig.tight_layout(rect=[0, 0, 1, 0.96])  # Deixa espaço para o título
                fig.savefig(os.path.join(plots_dir, 'boxplot_geral.png'), dpi=150, bbox_inches='tight')
                plt.close(fig)

        
        if 'stats' in dados['mediana']:
            fig, ax = plt.subplots(figsize=(12, 6))
            dados['mediana']['stats'].groupby('Name')['Average Response Time'].median().nlargest(30).plot(kind='barh', ax=ax, color='steelblue')
            ax.set_title(f'Tempo Mediano por Endpoint - {cenario}')
            ax.set_xlabel('ms')
            ax.grid(True, alpha=0.3)
            fig.tight_layout()
            fig.savefig(os.path.join(plots_dir, 'endpoints_mediana.png'), dpi=150)
            plt.close(fig)


In [12]:
path_csv_files = "results/*/*/results_stats.csv"
csv_files = glob.glob(path_csv_files)
resultados = []

for file_path in csv_files:
    df_stats = pd.read_csv(file_path)
    partes = file_path.split('\\')
    cenario = partes[1] if len(partes) > 1 else 'Unknown'
    repeticao = partes[2] if len(partes) > 2 else 'Unknown'
    linha_agregada = df_stats[df_stats['Name'] == 'Aggregated']
    
    if not linha_agregada.empty:
        linha = linha_agregada.iloc[0]
        
        resultado = {
            'Cenário': cenario,
            'Repetição': repeticao,
            'Total Requests': linha['Request Count'],
            'Total Failures': linha['Failure Count'],
            'Requests/s': linha['Requests/s'],
            'Failures/s': linha['Failures/s'],
            'Avg Response Time (ms)': linha['Average Response Time'],
            'Min Response Time (ms)': linha['Min Response Time'],
            'Max Response Time (ms)': linha['Max Response Time'],
            'Median Response Time (ms)': linha['Median Response Time'],
            'p50 (ms)': linha['50%'],
            'p66 (ms)': linha['66%'],
            'p75 (ms)': linha['75%'],
            'p80 (ms)': linha['80%'],
            'p90 (ms)': linha['90%'],
            'p95 (ms)': linha['95%'],
            'p98 (ms)': linha['98%'],
            'p99 (ms)': linha['99%'],
            'p99.9 (ms)': linha['99.9%'],
            'p99.99 (ms)': linha['99.99%'],
            'p100 (ms)': linha['100%']
        }
        
        resultados.append(resultado)

df_resumo = pd.DataFrame(resultados)
df_resumo = df_resumo.sort_values(['Cenário', 'Repetição'])
display(df_resumo)

df_cenarios = df_resumo.groupby('Cenário').agg({
    'Total Requests': ['mean', 'std', 'min', 'max'],
    'Total Failures': ['mean', 'std', 'min', 'max'],
    'Requests/s': ['mean', 'std', 'min', 'max'],
    'Avg Response Time (ms)': ['mean', 'std', 'min', 'max'],
    'p50 (ms)': ['mean', 'std', 'min', 'max'],
    'p95 (ms)': ['mean', 'std', 'min', 'max'],
    'p99 (ms)': ['mean', 'std', 'min', 'max']
}).round(2)
display(df_cenarios)

df_resumo.to_csv(os.path.join(path_results, 'resumo_resultados.csv'), index=False)
df_cenarios.to_csv(os.path.join(path_results, 'resumo_cenarios.csv'))

,Cenário,Repetição,Total Requests,Total Failures,Requests/s,Failures/s,Avg Response Time (ms),Min Response Time (ms),Max Response Time (ms),Median Response Time (ms),...,p66 (ms),p75 (ms),p80 (ms),p90 (ms),p95 (ms),p98 (ms),p99 (ms),p99.9 (ms),p99.99 (ms),p100 (ms)
0,CenarioA,rep1,14711,0,24.545970,0.000000,25.342290,2.9177,340.5748,12.0,...,30,44,48,59,70,88,120,210,250,340
1,CenarioA,rep10,14748,0,24.597199,0.000000,24.650462,2.9189,292.5069,12.0,...,31,44,47,56,68,82,110,170,230,290
2,CenarioA,rep2,14722,0,24.578246,0.000000,23.898679,2.8634,481.7591,12.0,...,29,41,47,54,65,80,98,200,470,480
3,CenarioA,rep3,14675,0,24.504659,0.000000,23.670323,2.9246,307.7694,12.0,...,28,41,47,54,65,78,98,180,310,310
4,CenarioA,rep4,14657,0,24.477479,0.000000,24.538323,3.1088,304.7224,11.0,...,29,41,47,55,70,89,120,220,300,300
5,CenarioA,rep5,14712,0,24.562115,0.000000,24.697745,2.9714,277.0747,12.0,...,30,44,48,57,68,83,100,180,280,280
6,CenarioA,rep6,14669,0,24.493613,0.000000,25.711153,2.8522,474.1542,12.0,...,31,45,48,59,72,91,120,200,460,470
7,CenarioA,rep7,14775,0,24.656745,0.000000,25.648334,2.8187,331.9409,11.0,...,32,45,48,61,74,93,110,180,320,330
8,CenarioA,rep8,14706,0,24.555165,0.000000,23.483004,2.9596,333.7056,11.0,...,26,38,46,55,69,85,110,180,330,330
9,CenarioA,rep9,14740,0,24.590272,0.000000,25.746235,3.0588,506.7512,12.0,...,31,44,48,58,72,93,120,250,490,510


Total Requests                       Total Failures                   \
                   mean     std    min    max           mean     std min  max   
Cenário                                                                         
CenarioA        14711.5   37.16  14657  14775            0.0    0.00   0    0   
CenarioB        28604.3  123.47  28450  28839            0.0    0.00   0    0   
CenarioC        22372.8  329.51  21882  22796          107.9  212.62   0  586   

         Requests/s        ... p50 (ms)     p95 (ms)                      \
               mean   std  ...      min max     mean     std   min   max   
Cenário                    ...                                             
CenarioA      24.56  0.05  ...       11  12     69.3    2.95    65    74   
CenarioB      47.73  0.20  ...       13  16    346.0   48.12   260   400   
CenarioC      74.83  1.12  ...       29  40   3090.0  251.44  2600  3400   

         p99 (ms)                       
             mean      std   min   max  
Cenário                                 
CenarioA    110.6     9.34    98   120  
CenarioB    816.0   161.60   580  1100  
CenarioC   5400.0  1308.09  4300  8300  

[3 rows x 28 columns]

In [9]:
plots_dir = os.path.join(path_results, 'plots', 'comparativos')
os.makedirs(plots_dir, exist_ok=True)

# 1. COMPARAÇÃO DE THROUGHPUT (Requests/s)
fig, ax = plt.subplots(figsize=(10, 6))

throughput_stats = df_resumo.groupby('Cenário')['Requests/s'].agg(['mean', 'std'])
ax.bar(range(len(throughput_stats)), throughput_stats['mean'], 
       yerr=throughput_stats['std'], capsize=5, alpha=0.7, color='steelblue')
ax.set_xticks(range(len(throughput_stats)))
ax.set_xticklabels(throughput_stats.index, rotation=45)
ax.set_title('Throughput Médio por Cenário', fontsize=14)
ax.set_ylabel('Requests/s (média ± std)')
ax.grid(True, alpha=0.3, axis='y')

fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '01_throughput_comparison.png'), dpi=150, bbox_inches='tight')
plt.close(fig)

# 2. COMPARAÇÃO DE LATÊNCIA (Response Time)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Tempo médio de resposta
avg_stats = df_resumo.groupby('Cenário')['Avg Response Time (ms)'].agg(['mean', 'std'])
axes[0].bar(range(len(avg_stats)), avg_stats['mean'], 
            yerr=avg_stats['std'], capsize=5, alpha=0.7, color='mediumseagreen')
axes[0].set_xticks(range(len(avg_stats)))
axes[0].set_xticklabels(avg_stats.index, rotation=45)
axes[0].set_title('Tempo Médio de Resposta')
axes[0].set_ylabel('ms (média ± std)')
axes[0].grid(True, alpha=0.3, axis='y')

# p50 (Mediana)
p50_stats = df_resumo.groupby('Cenário')['p50 (ms)'].agg(['mean', 'std'])
axes[1].bar(range(len(p50_stats)), p50_stats['mean'], 
            yerr=p50_stats['std'], capsize=5, alpha=0.7, color='mediumpurple')
axes[1].set_xticks(range(len(p50_stats)))
axes[1].set_xticklabels(p50_stats.index, rotation=45)
axes[1].set_title('Latência p50 (Mediana)')
axes[1].set_ylabel('ms (média ± std)')
axes[1].grid(True, alpha=0.3, axis='y')

fig.suptitle('Análise de Latência por Cenário', fontsize=16, y=1.02)
fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '02_latency_comparison.png'), dpi=150, bbox_inches='tight')
plt.close(fig)

# 3. PERCENTIS DE LATÊNCIA (TODOS)
fig, ax = plt.subplots(figsize=(14, 8))

percentis = ['p50 (ms)', 'p66 (ms)', 'p75 (ms)', 'p80 (ms)', 'p90 (ms)', 'p95 (ms)', 'p98 (ms)', 'p99 (ms)']
cenarios = df_resumo['Cenário'].unique()
x = range(len(percentis))
width = 0.8 / len(cenarios)

for i, cenario in enumerate(cenarios):
    dados_cenario = df_resumo[df_resumo['Cenário'] == cenario][percentis].mean()
    ax.bar([p + i*width for p in x], dados_cenario, width=width, label=cenario, alpha=0.8)

ax.set_xticks([p + width*(len(cenarios)-1)/2 for p in x])
ax.set_xticklabels([p.replace(' (ms)', '') for p in percentis], rotation=45)
ax.set_ylabel('Tempo de Resposta (ms)')
ax.set_title('Distribuição de Percentis de Latência por Cenário')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '03_percentiles_comparison.png'), dpi=150)
plt.close(fig)

# 4. TAXA DE FALHAS
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Calcula taxa de falhas
df_resumo['Failure Rate (%)'] = (df_resumo['Total Failures'] / df_resumo['Total Requests']) * 100

# Total de falhas
failure_stats = df_resumo.groupby('Cenário')['Total Failures'].agg(['mean', 'std']).fillna(0)
axes[0].bar(range(len(failure_stats)), failure_stats['mean'], 
            yerr=failure_stats['std'], capsize=5, alpha=0.7, color='coral')
axes[0].set_xticks(range(len(failure_stats)))
axes[0].set_xticklabels(failure_stats.index, rotation=45)
axes[0].set_title('Total de Falhas por Cenário')
axes[0].set_ylabel('Número de Falhas')
axes[0].set_ylim(bottom=0)  # Força escala a começar em 0
axes[0].grid(True, alpha=0.3, axis='y')

# Taxa de falhas (%)
failure_rate_stats = df_resumo.groupby('Cenário')['Failure Rate (%)'].agg(['mean', 'std']).fillna(0)
axes[1].bar(range(len(failure_rate_stats)), failure_rate_stats['mean'], 
            yerr=failure_rate_stats['std'], capsize=5, alpha=0.7, color='orangered')
axes[1].set_xticks(range(len(failure_rate_stats)))
axes[1].set_xticklabels(failure_rate_stats.index, rotation=45)
axes[1].set_title('Taxa de Falhas por Cenário')
axes[1].set_ylabel('Taxa de Falhas (%)')
axes[1].set_ylim(bottom=0)  # Força escala a começar em 0
axes[1].grid(True, alpha=0.3, axis='y')

fig.suptitle('Análise de Confiabilidade', fontsize=16, y=1.02)
fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '04_failure_analysis.png'), dpi=150, bbox_inches='tight')
plt.close(fig)

# 5. COMPARAÇÃO THROUGHPUT vs LATÊNCIA (Scatter)
fig, ax = plt.subplots(figsize=(12, 8))

for cenario in cenarios:
    dados = df_resumo[df_resumo['Cenário'] == cenario]
    ax.scatter(dados['Requests/s'], dados['Avg Response Time (ms)'], 
               label=cenario, s=100, alpha=0.6)

ax.set_xlabel('Throughput (Requests/s)', fontsize=12)
ax.set_ylabel('Latência Média (ms)', fontsize=12)
ax.set_title('Trade-off: Throughput vs Latência', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '05_throughput_vs_latency.png'), dpi=150)
plt.close(fig)


fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

metricas_radar = {
    'Throughput': 'Requests/s',
    'Latência p50': 'p50 (ms)',
    'Confiabilidade': 'Failure Rate (%)'
}

dados_norm = {}
for label, col in metricas_radar.items():
    valores = df_resumo.groupby('Cenário')[col].mean()
    
    val_min = valores.min()
    val_max = valores.max()
    val_range = val_max - val_min
    
    if val_range == 0:
        dados_norm[label] = pd.Series({c: 50 for c in valores.index})
    else:
        if 'Failure' in col or 'Latência' in label:
            dados_norm[label] = 100 - ((valores - val_min) / val_range * 80)
        else:
            dados_norm[label] = ((valores - val_min) / val_range * 80) + 20

categorias = list(metricas_radar.keys())
N = len(categorias)
angulos = [n / float(N) * 2 * pi for n in range(N)]
angulos += angulos[:1]

cenarios = df_resumo['Cenário'].unique()
for cenario in cenarios:
    valores = [dados_norm[cat][cenario] for cat in categorias]
    valores += valores[:1]
    ax.plot(angulos, valores, 'o-', linewidth=2, label=cenario, markersize=8)
    ax.fill(angulos, valores, alpha=0.15)

ax.set_xticks(angulos[:-1])
ax.set_xticklabels(categorias, fontsize=12)
ax.set_ylim(0, 100)
ax.set_yticks([20, 40, 60, 80, 100])
ax.set_title('Comparação Multidimensional de Performance\n(100 = Maior valor)', fontsize=14, pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.grid(True)
fig.tight_layout()
fig.savefig(os.path.join(plots_dir, '06_radar_comparison.png'), dpi=150, bbox_inches='tight')
plt.close(fig)